In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sciencehotpotqa-logical/sciencehotpot_logical_qa.csv


In [2]:
!pip install -q transformers accelerate bitsandbytes peft trl datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

os.environ['HF_TOKEN'] = user_secrets.get_secret("hugging_face")

In [4]:
from huggingface_hub import login
login(new_session=False)

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Load model in 4-bit for QLoRA
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_4bit=True,              # 4-bit quantization
    device_map="auto",              # Automatically split layers to GPU
    bnb_4bit_quant_type="nf4",      # NormalFloat4 (better for QLoRA)
    bnb_4bit_use_double_quant=True, # More compression
    torch_dtype="auto"
)

# Prepare for LoRA fine-tuning
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8,            # Rank
    lora_alpha=16,  # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

print("Model ready for QLoRA fine-tuning!")


2025-08-11 22:45:43.556682: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754952343.788330      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754952343.856536      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Model ready for QLoRA fine-tuning!


In [6]:
import pandas as pd
df = pd.read_csv('/kaggle/input/sciencehotpotqa-logical/sciencehotpot_logical_qa.csv')

In [7]:
df.head()

,question,correct_combinations,incorrect_combinations,correct_answers,incorrect_answers,original_context,all_sentences,document
0,Which of the following statements accurately d...,['Cadmium Chloride is slightly soluble in wate...,['Cadmium Chloride is highly soluble in alcoho...,['Cadmium Chloride is slightly soluble in wate...,['Cadmium Chloride is highly soluble in alcoho...,Cadmium Chloride is slightly soluble in this c...,['Cadmium chloride is a white crystalline comp...,Cadmium Chloride is slightly soluble in this c...
1,What characteristics do Kniphofia and Baptisia...,['Both Kniphofia and Baptisia produce flowerin...,['Kniphofia and Baptisia are classified as shr...,['Both Kniphofia and Baptisia produce flowerin...,['Kniphofia and Baptisia are both classified a...,What type of vegetation does Kniphofia and Bap...,['The Dinaric calcareous silver fir forests ar...,What type of vegetation does Kniphofia and Bap...
2,Which of the following statements best describ...,['Boreo-arctic Montane species are adapted to ...,['Boreo-arctic Montane species are primarily f...,['Boreo-arctic Montane species are typically f...,['Boreo-arctic Montane species are primarily f...,What type of species is a Boreo-arctic Montane...,"[""The five main latitude regions of the Earth'...",What type of species is a Boreo-arctic Montane...
3,What are the shared characteristics of Fotherg...,['Fothergilla is a genus of flowering plants t...,['Fothergilla and Clerodendrum are both native...,['Fothergilla and Clerodendrum both belong to ...,['Fothergilla and Clerodendrum are both native...,What type of vegetation does Fothergilla and C...,"['Clerodendrum infortunatum, the hill glory bo...",What type of vegetation does Fothergilla and C...
4,Which statements accurately reflect the classi...,['Dendranthema boreale is a flowering plant th...,['Dendranthema boreale is classified in the Or...,['Dendranthema boreale belongs to the Asterace...,['Dendranthema boreale is classified in the Or...,What are the plants in the same genus as the D...,['Navarretia is a genus of about 30 species of...,What are the plants in the same genus as the D...


In [8]:
import random
import ast
import pandas as pd
import numpy as np


def safe_eval(val):
    if isinstance(val, list):  # already list
        return val
    if pd.isna(val):  # NaN
        return []
    if isinstance(val, str):  # string that looks like a list
        return ast.literal_eval(val)
    return []  # fallback

df['correct_combinations'] = df['correct_combinations'].apply(safe_eval)
df['incorrect_combinations'] = df['incorrect_combinations'].apply(safe_eval)


qa_pairs = []
correct_label_cycle = ['A', 'B', 'C', 'D']
label_index = 0  # To cycle through correct answer positions


for idx, row in df.iterrows():
    if len(row['correct_combinations']) == 0 or len(row['incorrect_combinations']) == 0:
        continue  # skip if missing answers
    
    question = row['question']
    if len(row['correct_combinations']) > 1:
        correct_ans = row['correct_combinations'][1]
    else:
        correct_ans = row['correct_combinations'][0]
    
    incorrect_list = row['incorrect_combinations']
    if len(incorrect_list) >= 3:
        incorrect_ans = random.sample(incorrect_list, 3)
    else:
        incorrect_ans = random.choices(incorrect_list, k=3)
    
    # We'll fix the correct answer position according to label_index
    labels = ['A', 'B', 'C', 'D']
    correct_label = correct_label_cycle[label_index % 4]
    label_index += 1
    
    # Place correct answer in the position indicated by correct_label
    options = incorrect_ans.copy()
    options.insert(labels.index(correct_label), correct_ans)
    
    qa_pairs.append({
        'question': question,
        'A': options[0],
        'B': options[1],
        'C': options[2],
        'D': options[3],
        'correct_label': correct_label,
        'correct_answer_text': correct_ans
    })

qa_df = pd.DataFrame(qa_pairs)
print(f"Generated {len(qa_df)} QA pairs with balanced correct answer positions.")

qa_df.to_csv('sciencehotpotqa_pairs.csv',index=False)


Generated 1902 QA pairs with balanced correct answer positions.


In [9]:
qa_df.head()

,question,A,B,C,D,correct_label,correct_answer_text
0,Which of the following statements accurately d...,Ethanol has the chemical formula C2H5OH AND Ca...,Ethanol is known as ethyl alcohol AND Cadmium ...,Ethanol is a solid that is not hygroscopic AND...,Cadmium Chloride is highly soluble in alcohol ...,A,Ethanol has the chemical formula C2H5OH AND Ca...
1,What characteristics do Kniphofia and Baptisia...,Kniphofia and Baptisia are classified as shrub...,Both Kniphofia and Baptisia have flowers that ...,Both Kniphofia and Baptisia have flowers that ...,Both Kniphofia and Baptisia develop pods after...,B,Both Kniphofia and Baptisia have flowers that ...
2,Which of the following statements best describ...,The Arctic Circle is located in the southern h...,Boreo-arctic Montane species are not adapted t...,The Arctic Circle is the most northerly latitu...,Boreo-arctic Montane species are primarily fou...,C,The Arctic Circle is the most northerly latitu...
3,What are the shared characteristics of Fotherg...,Fothergilla grows predominantly in arid desert...,Fothergilla and Clerodendrum are both native t...,Fothergilla is categorized under the family Ha...,Both Fothergilla and Clerodendrum are classifi...,D,Both Fothergilla and Clerodendrum are classifi...
4,Which statements accurately reflect the classi...,"As a flowering plant in the Asteraceae family,...",Chrysanthemum is a genus in the Rosaceae famil...,Dendranthema boreale belongs to the Asteraceae...,Dendranthema boreale is classified in the Orch...,A,"As a flowering plant in the Asteraceae family,..."


In [10]:
def zero_shot_qa(model, tokenizer, qa_dataframe):
    """
    Performs zero-shot question answering on a DataFrame of multiple-choice questions,
    tokenizing the input correctly to avoid errors.

    Args:
        model: A language model compatible with the specified prompting format.
        tokenizer: The tokenizer for the language model.
        qa_dataframe: A pandas DataFrame with columns 'question', 'A', 'B', 'C', 'D',
                      and 'correct_label'.

    Returns:
        A list of dictionaries, where each dictionary contains the original question,
        the model's predicted label, and the correct label.
    """
    results = []
    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain the following columns: {required_cols}")
    
    # Define the set of valid answer choices for quick lookup
    valid_choices = {'A', 'B', 'C', 'D'}

    for idx, row in qa_dataframe.iterrows():
        predicted_label = None
        try:
            # Construct the prompt with a clear, strict instruction
            prompt = f"""Question:
{str(row['question'])}

Options:
A. {str(row['A'])}
B. {str(row['B'])}
C. {str(row['C'])}
D. {str(row['D'])}

Please respond with ONLY the single, capital letter (A, B, C, or D) that is the correct answer. Do not include any other text, punctuation, or explanation.
Answer:"""

            # 1. TOKENIZE the input prompt string
            inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
            inputs = {key: value.to(model.device) for key, value in inputs.items()}
            
            # 2. GENERATE the response using the tokenized input
            # We explicitly pass `input_ids` and `attention_mask`
            output_tokens = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=2,
                do_sample=False
            )
            
            # 3. DECODE the output tokens back into a string
            # We slice the output to get only the newly generated tokens
            generated_token_ids = output_tokens[0][inputs['input_ids'].shape[1]:]
            generated_text = tokenizer.decode(generated_token_ids, skip_special_tokens=True)
            
            # 4. Process the generated text for a strict answer
            # Strip all whitespace and common punctuation, then get the first character
            cleaned_text = generated_text.strip().upper().replace('.', '').replace(',', '').replace(' ', '')
            
            # Validate that the first character is a valid choice
            if cleaned_text and cleaned_text[0] in valid_choices:
                predicted_label = cleaned_text[0]

        except Exception as e:
            # Catch any errors during processing and log them
            print(f"Error processing row {idx}: {e}")
        
        results.append({
            'question': str(row['question']),
            'predicted_label': predicted_label,
            'correct_label': str(row['correct_label']).strip().upper()
        })
            
    return results

    

# Usage example
results = zero_shot_qa(model,tokenizer, qa_df)
my_results = pd.DataFrame(results)
my_results.to_csv('sciencehotpotqa_zeroshot_results.csv',index=False)

for res in results[:20]:
    print(f"Q: {res['question']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more de

Q: Which of the following statements accurately describes the properties of Cadmium Chloride and Ethanol?
Predicted answer: A
Correct answer: A
-----
Q: What characteristics do Kniphofia and Baptisia share in their flowering plant structure?
Predicted answer: C
Correct answer: B
-----
Q: Which of the following statements best describes the characteristics of Boreo-arctic Montane species and their habitat?
Predicted answer: C
Correct answer: C
-----
Q: What are the shared characteristics of Fothergilla and Clerodendrum regarding their classification and habitat?
Predicted answer: D
Correct answer: D
-----
Q: Which statements accurately reflect the classifications and characteristics of the plant Dendranthema boreale?
Predicted answer: A
Correct answer: A
-----
Q: Which of the following statements accurately describes the properties of sulfuric acid and its reaction with phenazine?
Predicted answer: B
Correct answer: B
-----
Q: Which of the following statements correctly describes the ch

In [11]:
my_results.head()

,question,predicted_label,correct_label
0,Which of the following statements accurately d...,A,A
1,What characteristics do Kniphofia and Baptisia...,C,B
2,Which of the following statements best describ...,C,C
3,What are the shared characteristics of Fotherg...,D,D
4,Which statements accurately reflect the classi...,A,A
